# Load Dialo

In [52]:
import numpy as np
import json
file_in = 'linked.json'
with open(file_in, "r") as input_file:
    input_data = json.load(input_file)

#print( len(input_data))
n = len(input_data)
#n = 20  # Replace 20 with your desired value of n
num_values = 11

# Generate 10 evenly spaced numbers ranging from 1 to n
evenly_spaced_numbers = list(np.linspace(1, n, num_values, dtype=int))
evenly_spaced_numbers.pop(-1)
print(evenly_spaced_numbers)

for i in range(500):
    try:
        print(f"{evenly_spaced_numbers.index(i)*10:.0f}%")
    except:
        None
#print(i)

[1, 122, 243, 364, 486, 607, 728, 850, 971, 1092]
0%
10%
20%
30%
40%


In [1]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
print('model loaded')

model loaded


# Use Dialo

In [7]:
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


>> User: How is the weather today?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: It's been raining for the past few days.


>> User: Hey!


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not seeing it.


>> User: I'll look into it.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not seeing it either.


In [ ]:
#move all processes to GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

def chat(model, prompt, user_input):
    input_text = f"Prompt: {prompt}\nUser: {user_input}\nModel:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    #input_ids = input_ids.to('cuda')
    response_ids = model.generate(input_ids, max_length=50, num_return_sequences=1)
    response = tokenizer.decode(response_ids[0], skip_special_tokens=True)
    return response

# Load training data and create trainer

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import json
# Load pre-trained chat model and tokenizer (e.g., gpt-3.5-turbo)

# Prepare your training data (include prompts, user inputs, and responses)
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="sequentialised.json",
    #file_path="train_data.json",
    block_size=128,  # Adjust block size if needed
)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Fine-tuning configuration
training_args = TrainingArguments(
    output_dir="./chatbot-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_device_train_batch_size=32,
    save_steps=2,
    logging_steps=1,
    max_steps = 20
)

# Create Trainer and fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

print("trainer created")

trainer created


# Run trainer

In [4]:
trainer.train()
trainer.save_model()
print("fine-tuned model saved into ./chatbot-finetuned")

Step,Training Loss
1,7.619800
2,6.010000
3,5.099600
4,4.234000
5,3.790700
6,3.316000
7,3.140800
8,2.789200
9,2.837600
10,2.624200


fine-tuned model saved into ./chatbot-finetuned


# Load fine-tuned model

In [8]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
model_name = "./chatbot-finetuned/checkpoint-20/"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)
model_ft = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config
    , trust_remote_code=True
)

In [9]:
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model_ft.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


>> User: How is the weather today?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: It's been raining for a few days now.


>> User: Hey!


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not seeing it.


>> User: I'll look into it.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not seeing it either.
